In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # Import matplotlib for plotting
import yfinance as yf

In [ ]:
df = yf.download('BTC-USD', start="2018-01-01", end="2022-12-01")
# Flatten the columns if they are MultiIndex
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)
df=df.reset_index()
print(type(df['Close']))
print(df.columns)

[*********************100%***********************]  1 of 1 completed

<class 'pandas.core.series.Series'>
Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')


In [ ]:
#indicator functions
def SMA(series, window):
    return series.rolling(window=window).mean()

def MACD(series, fast=12, slow=26, signal=9):
    fast_ema = series.ewm(span=fast, adjust=False).mean()
    slow_ema = series.ewm(span=slow, adjust=False).mean()
    macd_line = fast_ema - slow_ema
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    return macd_line, signal_line

def RSI(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    #handle division by zero in RSI calculation
    rs = rs.replace([np.inf, -np.inf], np.nan)
    return 100 - (100 / (1 + rs))

In [ ]:
#signal generation
df = df.copy()
# Indicators
df['sma_fast'] = SMA(df['Close'], 10)
df['sma_slow'] = SMA(df['Close'], 30)

macd_line, signal_line = MACD(df['Close'])
df['macd'] = macd_line
df['macd_signal'] = signal_line
df['rsi'] = RSI(df['Close'])

df.dropna(inplace=True)

#trading signals
df['signal'] = 0
df.loc[
 (df['sma_fast'] > df['sma_slow']) &
 (df['macd'] > df['macd_signal']) &
 (df['rsi'] < 70), 'signal'
] = 1  #long

df.loc[
 (df['sma_fast'] < df['sma_slow']) &
 (df['macd'] < df['macd_signal']) &
 (df['rsi'] > 30), 'signal'
] = -1  #short

In [ ]:
def backtest(df, stop_loss_pct=0.02):
    trades = []
    position = 0
    entry_price = 0
    entry_time = None

    for i in range(1, len(df)):

        if i > 0:
            previous_signal = df.iloc[i-1]['signal']
        else:
            previous_signal = 0 #assume no signal at the very start

        current_signal = df.iloc[i]['signal']
        current_close = df.iloc[i]['Close']
        current_time = df.iloc[i]['Date']


        if position == 0:
            if current_signal == 1:
                position = 1
                entry_price = current_close
                entry_time = current_time
                entry_idx = i
            elif current_signal == -1:
                position = -1
                entry_price = current_close
                entry_time = current_time
                entry_idx = i

        elif position == 1:
            #stop loss breach
            if current_close < entry_price * (1 - stop_loss_pct):
                trades.append({'entry': entry_price, 'exit': current_close, 'side': 'long', 'entry_time': entry_time, 'exit_time': current_time, 'exit_idx':i,'entry_idx':entry_idx})
                position = 0
                entry_idx = None
                exit_idx = None
            #squaring off
            elif current_signal == -1 or (current_signal == 0 and previous_signal == 1):
                 trades.append({'entry': entry_price, 'exit': current_close, 'side': 'long', 'entry_time': entry_time, 'exit_time': current_time, 'exit_idx':i,'entry_idx':entry_idx})
                 position = 0
                 entry_idx = None
                 exit_idx = None
                 #if new signal is -1, enter short immediately
                 if current_signal == -1:
                     position = -1
                     entry_price = current_close
                     entry_time = current_time
                     entry_idx = i

        elif position == -1:
            #stop loss breach
            if current_close > entry_price * (1 + stop_loss_pct):
                trades.append({'entry': entry_price, 'exit': current_close, 'side': 'short', 'entry_time': entry_time, 'exit_time': current_time, 'exit_idx':i,'entry_idx':entry_idx})
                position = 0
                entry_idx = None
                exit_idx = None

            #squaring off
            elif current_signal == 1 or (current_signal == 0 and previous_signal == -1):
                trades.append({'entry': entry_price, 'exit': current_close, 'side': 'short', 'entry_time': entry_time, 'exit_time': current_time, 'exit_idx':i,'entry_idx':entry_idx})
                position = 0
                entry_idx = None
                exit_idx = None
                #if new signal is 1, enter long immediately
                if current_signal == 1:
                    position = 1
                    entry_price = current_close
                    entry_time = current_time
                    entry_idx = i


    #closing last position (if open)
    if position != 0 and len(df) > 0:
         # Ensure there's data to get the last close and datetime
        last_close = df.iloc[-1]['Close']
        last_time = df.iloc[-1]['Date']
        trades.append({'entry': entry_price, 'exit': last_close, 'side': 'long' if position == 1 else 'short', 'entry_time': entry_time, 'exit_time': last_time, 'exit_idx':i,'entry_idx':entry_idx})


    trades_df = pd.DataFrame(trades)
    trades_df['pnl'] = 0.0
    trades_df['max_dip'] = 0.0
    trades_df['max_drawdown'] = 0.0
    trades_df['trade_duration'] = trades_df['exit_time'] - trades_df['entry_time']

    return trades_df

In [ ]:
initial_investment= 100000
risk_free_profit= 0.02

In [ ]:
def metrics(df, trades):

  metric={}
  for i in range(len(trades)):
    if trades['side'].iloc[i]=='long':
      trades['pnl'].iloc[i]= (trades['exit'].iloc[i] - trades['entry'].iloc[i])/trades['entry'].iloc[i] *initial_investment
    else:
      trades['pnl'].iloc[i]= (trades['entry'].iloc[i] - trades['exit'].iloc[i])/trades['entry'].iloc[i] *initial_investment

  for i in range(1,len(trades)-1):
    minima= df['Close'].iloc[trades['exit_idx'].iloc[i-1]]
    peak= df['Close'].iloc[trades['exit_idx'].iloc[i-1]]
    max_drawdown=0
    drawdown_pct=0

    for j in range(trades['exit_idx'].iloc[i-1] +1, trades['entry_idx'].iloc[i+1] +1):
      if j< len(trades):
        if df['Close'].iloc[j]<minima:
          minima= df['Close'].iloc[j]
    trades['max_dip'].iloc[i]= (df['Close'].iloc[trades['entry_idx'].iloc[i]] - minima)/df['Close'].iloc[trades['entry_idx'].iloc[i]] *100

    for j in range(trades['exit_idx'].iloc[i-1] +1, trades['entry_idx'].iloc[i+1] +1):
      if j<len(trades):
        if df['Close'].iloc[j]>peak:
          peak= df['Close'].iloc[j]
          drawdown_pct = (peak - df['Close'].iloc[j]) * 100 / peak
        if drawdown_pct > max_drawdown:
          max_drawdown = drawdown_pct

    trades['max_drawdown'].iloc[i]= max_drawdown

  net_profit_pct= (trades['pnl'].sum())/initial_investment *100
  metric['Net Profit %']= net_profit_pct

  gross_profit_pct= (trades['pnl'][trades['pnl']>0].sum())/initial_investment *100
  metric['Gross Profit %']= gross_profit_pct

  metric['Total Trades']= len(trades)
  metric['Wins']= len(trades[trades['pnl']>0])
  metric['Losses']= len(trades[trades['pnl']<0])
  metric['Win Rate']= len(trades[trades['pnl']>0])/len(trades)*100
  metric['Avg Trade Duration']= trades['trade_duration'].mean()
  metric['Max Trade Duration']= trades['trade_duration'].max()
  metric['Max Drawdown']= trades['max_drawdown'].max()
  metric['Avg Drawdown']= trades['max_drawdown'].mean()
  metric['Max Dip']= trades['max_dip'].max()
  metric['Avg Dip']= trades['max_dip'].mean()

  if trades['pnl'].std()>0:
    metric['Sharpe Ratio']= (trades['pnl'].sum()-risk_free_profit)/trades['pnl'].std()
  else:
    metric['Sharpe Ratio']= np.nan

  if trades[trades['pnl']<0]['pnl'].std()>0:
    metric['Sortino Ratio']= (trades['pnl'].sum()-risk_free_profit)/trades[trades['pnl']<0]['pnl'].std()
  else:
    metric['Sortino Ratio']= np.nan

  return metric

In [ ]:
trades= backtest(df)
metrics= metrics(df, trades)

In [ ]:
cleaned_metrics = {
    key: float(value) if isinstance(value, np.floating) else int(value) if isinstance(value, np.integer) else value
    for key, value in metrics.items()
}

In [ ]:
print("Strategy Performance Metrics")
for key, value in cleaned_metrics.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

Strategy Performance Metrics
Net Profit %: 59.68
Gross Profit %: 430.06
Total Trades: 183
Wins: 82
Losses: 101
Win Rate: 44.81
Avg Trade Duration: 2 days 21:54:05.901639344
Max Trade Duration: 24 days 00:00:00
Max Drawdown: 0.00
Avg Drawdown: 0.00
Max Dip: 34.49
Avg Dip: 1.80
Sharpe Ratio: 10.13
Sortino Ratio: 21.05
